In [30]:
# This defines a function that will generate all possible amino acid combinations and their combined percent identity
# It will return a dataframe with all 190 amino acid combos as columns and the combined percent identities as rows

import numpy as np, pandas as pd, protfasta

# You need to input a multiple sequence alignment (MSA) fasta file, as well as a start and end position to take a slice of the MSA
def AllPossibleResidueCombos(alignment_file, start_position, end_position):

    # Read in fasta file
    fasta_file = protfasta.read_fasta(alignment_file, invalid_sequence_action = 'ignore', alignment = True, expect_unique_header = False)

    # Convert the alignment into a dataframe
    fasta_file = pd.DataFrame(fasta_file.items(), columns = ['Name', 'Sequence'])

    # Extract the protein sequences only and split each character in the alignment into multiple different columns
    ManyColumnsDF = fasta_file['Sequence'].str.split('', expand=True)
    display(ManyColumnsDF)
    
    # Create an empty dataframe
    PercentIdentityDF = pd.DataFrame()

    # Create a for loop that updates the amino acid percent identities in each position of the sequence alignment
    for column in ManyColumnsDF.loc[:, start_position:end_position].columns:
        AAs = ManyColumnsDF[column].value_counts()
        #Create a dictionary with all of the amino acid percent identities initialized to 0
        AA_dict = {'A':0, 'C':0, 'D':0, 'E':0, 'F':0, 'G':0, 'H':0, 'I':0, 'K':0, 'L':0, 'M':0, 'N':0, 'P':0, 'Q':0, 'R':0, 'S':0, 'T':0, 'V':0, 'W':0, 'Y':0}
        # Iterate though each amino acid present in the column and match it with its key in the dict
        for AA in AAs.index.tolist():   # 'AAs.index.tolist()' finds unique amino acids that are present
            for key in AA_dict:         # in a position in the alignment. It then adds them once to a list
                if key == AA:
                    #After matching an amino acid to its key, the value count of the amino acid is used
                    # to calculate the percent identity of an amino acid in a specific position in the alignment
                    AA_dict[key] = AA_dict[key] + round(AAs[AA]/sum(AAs.values.tolist()) * 100)
                    PID_DF2 = pd.DataFrame(list(AA_dict.items()), columns = ['AA Code', column]).set_index('AA Code')
                    PercentIdentityDF[column] = PID_DF2[column]
                else:
                    continue

    # Take a specific section from the protein MSA
    TransposedDF = PercentIdentityDF.transpose()
    display(TransposedDF)
    
    # Remove the index for the column names row
    TransposedDF.columns.name = None

    # Calculate all possible combinations of pairs from each row
    all_residue_combinations = [combinations(row, 2) for _, row in TransposedDF.iterrows()]

    # Collect the row divisions in a list of dictionaries
    combos_data = []

    display(all_residue_combinations)
    
    # Iterate through all_residue_combinations to make a dict with the
    # residue pair as the key and the change in percent identity as the value
    for index, combinations_list in enumerate(all_residue_combinations):
        min_value = [min(comb) if sum(comb) != 0 else 0 for comb in combinations_list]
        column_names = [f'{col1}{col2}' for col1, col2 in combinations(TransposedDF.columns, 2)]
        combos_data.append(dict(zip(column_names, min_value)))

    # Create the final DataFrame all_residue_combos_df with the original index from MainDF
    all_possible_combos_df = pd.DataFrame(combos_data, index=TransposedDF.index)


    # Step 1: Identify the column with the maximum value for each row
    max_columns = TransposedDF.idxmax(axis=1)

    # Step 2: Extract the column names with the maximum values
    consensus_sequence = max_columns.apply(lambda col: col[0])

    # Step 3: Combine the extracted column names to form the consensus sequence
    consensus_sequence_string = ''.join(consensus_sequence)


    return consensus_sequence_string, all_possible_combos_df

In [31]:
# Write out the directory of the MSA file
gcn4_msa = "../data/zoonomia_toga_mca/all_AD_alignments/ABRAXAS1_121-200.fasta"

# Give color names to specific hex codes
aquamarine, light_blue, blue, green, light_gray, orange, pink, purple, white, mustard_yellow = "#00CCCC", "#54A0EB", "#0B27B5", '#09B328', "#D3D3D3", "#FFAA00", "#FF99C2", "#BA34EB", "#FFFFFF", "#DECC09"

# Set the custom color scheme for amino acid pairs. All other pairs not specified here will be the color gray
color_scheme = {
    'DE': 'red',
    'FL': purple,
    'FY': mustard_yellow,
    'HQ': 'green',
    'IV': 'brown',
    'ST': light_blue
}


# ["AD", "AP", "AS", "DE", "DN", "DQ", "DS", "FL", "FY", "GS", "LM", "MP", "PS", "ST"]

In [32]:
import matplotlib.pyplot as plt, re
from itertools import combinations

#start = 2150

# Create a data frame that generates all possible amino acid combinations and their combined percent identity
consensus_sequence, all_residue_combos_df = AllPossibleResidueCombos(gcn4_msa, 1, start+200) # <-- Adjust start and end positions
all_residue_combos_df = all_residue_combos_df[all_residue_combos_df != 0]
display(all_residue_combos_df)
print("hi")
#motifs = ["DE"]






# --- Do NOT change the code below ---

# Set the size of the figure
plt.figure(figsize=(24, 10))



# Find the overall maximum value in the DataFrame
overall_max_value = all_residue_combos_df.max().max()

# # Find the locations of each motif
# for motif in motifs:
#     matches = re.finditer(motif, consensus_sequence)
#     motif_coords = [(match.start(), match.end()) for match in matches]

#     # After achieving this, count the number of motifs
#     num_motifs = len(motif_coords)
#     print(f"Number of {motif} motifs: {num_motifs}")

#     for coord in motif_coords:
#         start_indx, end_indx = coord[0], coord[1] - 1
#         amino_acid_list = list(consensus_sequence[start_indx:end_indx])

#         # Calculate the coordinates for the label
#         label_x = start + (start_indx + end_indx) / 2
#         label_y = overall_max_value + 2  # Adjust the height of the label above the highest point

#         # Create a line segment using plot
#         plt.plot([start + start_indx, start + end_indx], [overall_max_value + 1, overall_max_value + 1], color='black')

#         # Add a label using annotate with a text parameter
#         plt.annotate(motif, xy=(label_x, label_y), xytext=(label_x, label_y), ha='center', va='bottom')



# # Plot each point and use the color code defined in color_scheme
# for col in all_residue_combos_df.columns:
#     if col in color_scheme.keys():
#         plt.scatter(all_residue_combos_df.index, all_residue_combos_df[col], label=col, color=color_scheme[col], s=50)
#     else:
#         plt.scatter(all_residue_combos_df.index, all_residue_combos_df[col], color="gray", s=50, alpha=0.25)

# # Set x-axis ticks to every 5 units
# plt.xticks(range(min(all_residue_combos_df.index), max(all_residue_combos_df.index) + 1, 20))

# # Set ylim range
# plt.ylim(0, overall_max_value + 4)

# # Write out labels
# plt.title("Change in Percent Identity in each MSA Position")
# plt.xlabel("MSA Position")
# plt.ylabel("Change in Percent Identity")

# # Create a legend based on what user inputted into the variable called color_scheme
# legend_elements = [plt.Line2D([0], [0], marker='o', color='w', label=key, markerfacecolor=value, markersize=10) for key, value in color_scheme.items()]
# plt.legend(handles=legend_elements, loc=(1.05, 0.9))

# # Removing the grid and change layout
# plt.grid(False)
# plt.tight_layout()
# ax = plt.gca()
# ax.spines['top'].set_visible(False)
# ax.spines['right'].set_visible(False)

# plt.show()


# print(f"Values for DE:\n{all_residue_combos_df[all_residue_combos_df['DE'] > 10]['DE']}\n")
# print(f"Values for FL:\n{all_residue_combos_df[all_residue_combos_df['FL'] > 10]['FL']}\n")
# print(f"Values for FY:\n{all_residue_combos_df[all_residue_combos_df['FY'] > 10]['FY']}\n")
# print(f"Values for IV:\n{all_residue_combos_df[all_residue_combos_df['IV'] > 10]['IV']}")

,0,1,2,3,4,5,6,7,8,9,...,72,73,74,75,76,77,78,79,80,81
0,,L,Q,E,H,F,S,N,Q,D,...,S,R,A,V,Q,T,H,S,S,
1,,L,Q,K,H,L,S,S,R,E,...,S,R,A,I,K,T,H,S,S,
2,,L,Q,E,H,F,S,N,H,E,...,S,R,A,V,K,T,H,N,S,
3,,L,R,K,H,L,S,N,Q,E,...,N,R,A,V,K,T,H,S,S,
4,,L,Q,E,Y,F,P,S,Q,D,...,S,R,A,V,K,T,H,S,S,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
478,,L,Q,Q,H,L,S,S,R,E,...,N,R,A,V,K,T,-,-,-,
479,,L,Q,Q,H,L,S,S,Q,E,...,S,R,A,V,K,T,H,S,S,
480,,L,Q,R,R,L,S,S,Q,E,...,-,-,-,-,-,-,-,-,S,
481,,L,T,R,I,L,G,V,P,D,...,-,-,-,-,-,-,-,-,-,


AA Code,A,C,D,E,F,G,H,I,K,L,M,N,P,Q,R,S,T,V,W,Y
1,0,0,0,0,0,0,0,0,0,97,0,0,0,0,0,0,0,0,0,0
2,0,0,0,1,0,0,0,0,0,0,0,0,0,94,1,0,1,0,0,0
3,0,0,0,10,0,0,1,0,23,0,1,11,0,39,3,0,8,0,0,0
4,0,0,0,0,0,0,91,1,0,0,0,0,0,1,2,0,0,0,0,3
5,0,0,0,0,16,0,0,0,0,82,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,0,0,0,4,0,0,0,0,72,0,0,2,0,8,10,1,0,0,0,0
77,5,0,0,0,0,0,0,1,1,0,0,0,0,0,1,0,88,0,0,0
78,0,0,0,0,0,0,96,0,0,0,0,0,0,0,0,0,0,0,0,1
79,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,94,0,0,0,0


,AC,AD,AE,AF,AG,AH,AI,AK,AL,AM,...,ST,SV,SW,SY,TV,TW,TY,VW,VY,WY
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
77,NaN,NaN,NaN,NaN,NaN,NaN,1.0,1.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
78,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
79,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


hi


<Figure size 2400x1000 with 0 Axes>

In [21]:
# Filter out amino acid pairs based on y-axis values greater than the threshold
threshold = 15 # <-- change this number to test various thresholds




# --- Do NOT change the code below, only change the threshold above ---

# Filter out the dataframe based on the set threshold
filtered_combos = all_residue_combos_df[all_residue_combos_df > threshold]

# Convert the DataFrame to a new format with the index, amino acid pair, and value
filtered_data = filtered_combos.stack().reset_index()
filtered_data.columns = ['Index', 'Amino Acid Pair', 'Value']

# Sort the values so that they go from largest to smallest
filtered_data.sort_values('Value', ascending=False)

# Get the counts of each amino acid pair from the filtered_data DataFrame
amino_acid_pair_counts = filtered_data['Amino Acid Pair'].value_counts()

print(f"The following amino acid pairs, when added together, have a percent identity change that is greater than {threshold}.\nEach pair is accompanied by its respective count:\n\nPair: Count:\n{amino_acid_pair_counts.head(40)}")

The following amino acid pairs, when added together, have a percent identity change that is greater than 15.
Each pair is accompanied by its respective count:

Pair: Count:
Series([], Name: count, dtype: int64)


In [ ]:
import pandas as pd, warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

from scipy.stats import ttest_ind

for residue_pair in all_residue_combos_df.columns:

    # Extract the residue pair column
    residue_pair_column = all_residue_combos_df[residue_pair]

    # Filter non-zero values
    residue_interchange = residue_pair_column.fillna(0)
    residue_interchange = residue_interchange[residue_interchange != 0]
#     print(f"- Mean {residue_pair} Interchange: {residue_interchange.mean()}")

    # Extract all other columns
    other_residue_pairs = all_residue_combos_df.drop(residue_pair, axis=1)

    # Filter non-zero values in other columns
    other_residue_interchange = other_residue_pairs[other_residue_pairs != 0].stack()
#     print(f"- Mean Interchange of All Other Pairs: {other_residue_interchange.mean()}")

    # Perform independent two-sample t-test
    t_statistic, p_value = ttest_ind(residue_interchange, other_residue_interchange)
#     print(f"- t statistic: {t_statistic}\n- p value: {p_value}")

    # Check if the p-value is less than a significance level (e.g., 0.05)
    significance_level = 0.01
    if p_value < significance_level:
        print(f"Residue Pair: {residue_pair}")
        print(f"- Mean {residue_pair} Interchange: {residue_interchange.mean()}")
        print(f"- Mean Interchange of All Other Pairs: {other_residue_interchange.mean()}")
        print(f"- t statistic: {t_statistic}\n- p value: {p_value}")
        print(f"Result: The average values in the {residue_pair} column are statistically significant.\n\n")
#     else:
#         print(f"Result: No statistically significant difference found for {residue_pair}.\n\n")

# sum(all_residue_combos_df["DE"][all_residue_combos_df["DE"] != 0])/len(all_residue_combos_df["DE"][all_residue_combos_df["DE"] != 0])

Residue Pair: DE
- Mean DE Interchange: 6.39622641509434
- Mean Interchange of All Other Pairs: 2.540041067761807
- t statistic: 9.238599074329597
- p value: 3.782176931623634e-20
Result: The average values in the DE column are statistically significant.


Residue Pair: DN
- Mean DN Interchange: 4.02
- Mean Interchange of All Other Pairs: 2.5697674418604652
- t statistic: 3.3480316470913865
- p value: 0.0008206628107409514
Result: The average values in the DN column are statistically significant.


Residue Pair: DQ
- Mean DQ Interchange: 4.05
- Mean Interchange of All Other Pairs: 2.572793448589627
- t statistic: 3.05309562019679
- p value: 0.002278369786158409
Result: The average values in the DQ column are statistically significant.


Residue Pair: FL
- Mean FL Interchange: 4.827586206896552
- Mean Interchange of All Other Pairs: 2.5713637395053324
- t statistic: 3.9784190607364187
- p value: 7.048239124869648e-05
Result: The average values in the FL column are statistically signific

In [ ]:
import pandas as pd
from scipy.stats import ttest_ind

for residue_pair in color_scheme.keys():

    print(f"Checking for statistical significant interchange in {residue_pair} column...")

    # Extract the residue pair column
    residue_pair_column = all_residue_combos_df[residue_pair]

    total, counter = 0, 0
    for number in residue_pair_column:
        if number != 0:
#             print(row)
            total += number
            counter += 1

    residue_interchange = total / counter
    print(residue_interchange)

    # Extract all other columns
    other_residue_pairs = all_residue_combos_df.drop(residue_pair, axis=1)

    total_2, counter_2 = 0, 0
    for column in other_residue_pairs.columns:
        for number in other_residue_pairs[column]:
            if number != 0:
                total_2 += number
                counter_2 += 1

    other_residue_interchange = total_2 / counter_2
    print(other_residue_interchange)

    # Perform independent two-sample t-test
    t_statistic, p_value = ttest_ind(residue_interchange, other_residue_interchange)
    print(f"t statistic: {t_statistic}\np value: {p_value}")


#     print(filtered_residue_pair_column.mean(), other_columns.mean(axis=1))


    # Check if the p-value is less than a significance level (e.g., 0.05)
    significance_level = 0.05
    if p_value < significance_level:
        print(f"The average values in the {residue_pair} column are statistically significant.\n\n")
    else:
        print(f"No statistically significant difference found for {residue_pair}.\n\n")

Checking for statistical significant interchange in DE column...
nan
nan
t statistic: nan
p value: nan
No statistically significant difference found for DE.


Checking for statistical significant interchange in FL column...
nan
nan
t statistic: nan
p value: nan
No statistically significant difference found for FL.


Checking for statistical significant interchange in FY column...
nan
nan
t statistic: nan
p value: nan
No statistically significant difference found for FY.


Checking for statistical significant interchange in HQ column...
nan
nan
t statistic: nan
p value: nan
No statistically significant difference found for HQ.


Checking for statistical significant interchange in IV column...
nan
nan
t statistic: nan
p value: nan
No statistically significant difference found for IV.


Checking for statistical significant interchange in ST column...
nan
nan
t statistic: nan
p value: nan
No statistically significant difference found for ST.


